# Capstone Project - The Battle of Neighborhoods (Week 1)
## Analyzing Geospatial Data using Foursquare and Python

__created by MaroDataScience__

## Problem Description

## Discussion of the Background

## Data Description

## How the Data is used to solve the problem